In [1]:
import sys
from pathlib import Path
from pprint import pprint

sys.path.append(str('/Users/kenichiyamashita/by_case/aoz_wrapper/aozfileoperation'))
sys.path.append(str('../'))
import cv2
import fitz
import numpy as np
import pdf_file_operation as pfo
import perpendicularlinesdetection.utils as utils
from colorama import Back, Fore, Style
from perpendicularlinesdetection import utils
from tqdm.notebook import tqdm

print(fitz.__doc__)



PyMuPDF 1.19.6: Python bindings for the MuPDF 1.19.0 library.
Version date: 2022-03-03 00:00:01.
Built for Python 3.7 on darwin (64-bit).



# パスの設定

In [2]:
SAVE_PATH = Path('/Volumes/GoogleDrive-107448067308560871230/マイドライブ/'
                 '07_aoz_cloud/■メニュー別開発ファイル/1_配筋検査支援メニュー/'
                 'No9_表のデータベース化精度強化/yamashita_dev/img')

reinetsu_data_path_list = list(
    Path('/Volumes/GoogleDrive-117051946152776398938/.shortcut-targets-by-id/'
         '14r-TRcZ12gyDHMDSihPQrLFDumDD1UTI/01_冷熱事業/03_データ/'
         '0_アノテーションデータ/3_表領域・切り出し線/pdf_copy').glob('*.pdf'))

# PyMuPDF Tutorial

In [1]:
import fitz
# print(f'a3 size: {fitz.paper_size("a3")}')
# print(f'a0 size: {fitz.paper_size("a0")}')
for i in list(fitz.paper_sizes().keys())[:]:
    print(f'{i}: {fitz.paper_size(i)}')

a0: (2384, 3370)
a1: (1684, 2384)
a10: (74, 105)
a2: (1191, 1684)
a3: (842, 1191)
a4: (595, 842)
a5: (420, 595)
a6: (298, 420)
a7: (210, 298)
a8: (147, 210)
a9: (105, 147)
b0: (2835, 4008)
b1: (2004, 2835)
b10: (88, 125)
b2: (1417, 2004)
b3: (1001, 1417)
b4: (709, 1001)
b5: (499, 709)
b6: (354, 499)
b7: (249, 354)
b8: (176, 249)
b9: (125, 176)
c0: (2599, 3677)
c1: (1837, 2599)
c10: (79, 113)
c2: (1298, 1837)
c3: (918, 1298)
c4: (649, 918)
c5: (459, 649)
c6: (323, 459)
c7: (230, 323)
c8: (162, 230)
c9: (113, 162)
card-4x6: (288, 432)
card-5x7: (360, 504)
commercial: (297, 684)
executive: (522, 756)
invoice: (396, 612)
ledger: (792, 1224)
legal: (612, 1008)
legal-13: (612, 936)
letter: (612, 792)
monarch: (279, 540)
tabloid-extra: (864, 1296)


In [4]:
# open Document
num = 0
# doc = fitz.Document(str(reinetsu_data_path_list[num]))
doc = fitz.open(str(reinetsu_data_path_list[num]))
page = doc[0]
print(type(page))
# ページを読み込む
page = doc.load_page(0)
# page size を取得
page_rect = page.rect
print(f'page.rect: {page_rect}')

<class 'fitz.fitz.Page'>
page.rect: Rect(0.0, 0.0, 1191.0, 842.0)


## pdf内のテキストデータを扱う

In [5]:
HANDLING_TEXT = False
if HANDLING_TEXT:
    page_rawdict = page.get_text('rawdict')
    page_dict = page.get_text('dict')
    # page_xml = page.get_text('xml')
    width, height = page_rawdict['width'], page_rawdict['height']

    print(f'width: {width}, height: {height}')
    blocks = page_dict['blocks']
    for i in blocks[:]:
        pprint(i)
        print('*'*80)

## PDFドキュメントから画像を作成する

In [6]:
CREATE_IMAGE = False
if CREATE_IMAGE:
    # pageから画像を生成
    pix = page.get_pixmap(alpha=False)
    # dpiを変更する
    pix1 = page.get_pixmap(dpi=200)
    print(f'pix1: {pix1.width}, {pix1.height}')
    # img sizeを変更する
    zoom_x = 2.0  # horizontal zoom
    zoom_y = 2.0  # vertical zoom
    mat = fitz.Matrix(zoom_x, zoom_y)  # zoom factor 2 in each dimension
    pix2 = page.get_pixmap(matrix=mat)  # use 'mat' instead of the identity matrix
    print(f'pix2: {pix2.width}, {pix2.height}')
    # save image
    pix1.save(SAVE_PATH / f'{Path(doc.name).stem}_1.png')
    pix2.save(SAVE_PATH / f'{Path(doc.name).stem}_2.png')

## 図面の抽出

In [7]:
COLOR = False
if COLOR:
    print(Fore.RED + 'some red text')
    print(Back.GREEN + 'and with a green background')
    print(Style.DIM + 'and in dim text')
    print(Style.RESET_ALL)
    print('back to normal now')

In [11]:
DRAWING = False

line_list = []
rect_list = []
quad_list = []
curve_list = []
if DRAWING:
    # 各object情報を抽出する
    paths = page.get_cdrawings()
    print(f'{Fore.BLUE}len(paths):{Style.RESET_ALL} {len(paths)}')
    for path in tqdm(paths[:]):
        for item in path['items']:
            if item[0] == 'l':
                line_list.append([item[1], item[2]])
                # pdfにlineを追加する場合
                # page.add_line_annot(item[1], item[2])
            elif item[0] == 're':
                rect_list.append(item[1])
            elif item[0] == 'qu':
                quad_list.append(item[1])
            elif item[0] == 'c':
                curve_list.append([item[1], item[2], item[3], item[4]])
            else:
                raise ValueError(f'item[0]: {item[0]}')

    print(f'line: {line_list[:2]}')
    print(f'rect: {rect_list[:2]}')
    print(f'quad: {quad_list[:2]}')
    print(f'curve_list: {curve_list}')
    # doc.save(SAVE_PATH / f'{Path(doc.name).stem}_3.pdf')
    # PDF内の画像を抽出する


In [12]:
# 既存のpdfにlineを上書きし、保存する
OVERRIDE = False
if OVERRIDE:
    for line in line_list[:5]:
        print(line)
        page.draw_line(line[0], line[1], color=(1, 0, 0))
    doc.save(SAVE_PATH / 'test.pdf')

In [10]:
# pageを画像に変換し取得したlineデータを画像に上書きし保存する
PROCESS = False
if PROCESS:
    # get image
    img = AccessPDF.pixel2np(page)
    cv2.imwrite(str(SAVE_PATH / f'{Path(doc.name).stem}.png'), img)
    img_width = img.shape[1]
    print(img.shape)
    copy_img = img.copy()
    # 各linesを取得する
    lines_dict = utils.judge_direction(line_list)
    x_lines = utils.convert_coordinates(lines_dict['x_lines'], img_width)
    y_lines = utils.convert_coordinates(lines_dict['y_lines'], img_width)
    other_lines = utils.convert_coordinates(lines_dict['other_lines'], img_width)
    point_list = utils.convert_coordinates(lines_dict['point_list'], img_width)
    # 線を可視化し、画像を保存する
    thickness = 1
    pfo.draw_lines_from_pdf(x_lines, copy_img, color=(0, 0, 255), thickness=thickness)
    pfo.draw_lines_from_pdf(y_lines, copy_img, color=(255, 0, 0), thickness=thickness)
    pfo.draw_lines_from_pdf(other_lines, copy_img, color=(0, 255, 0), thickness=thickness)
    cv2.imwrite(str(SAVE_PATH / f'{Path(doc.name).stem}_override.png'), copy_img)

(842, 1191, 3)


True

In [11]:
XREF = False
if XREF:
    # pdf内のオブジェクトの総数を取得する
    xref_length = doc.xref_length()
    print(f'xref_length: {xref_length}')
    for xref in range(xref_length):
        print(f'extract_image: {doc.extract_image(xref)}')
    # page内の画像を抽出する
    xref = doc.page_xref(0) # pno=0
    print(f'xref: {xref}')
    d = doc.extract_image(16)
    print(d)
    # img_list = page.get_images()
    # print(f'img_list: {img_list}\nimg_info: {img_info}')